In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn import linear_model

Hice esta funcion usando sklearn, pero no pude descifrar cómo hacer la última consigna de la prediccion con esta librería, por lo tanto más adelante lo intenté con statsmodels.  

In [4]:
#Con sklearn
#La funcion pide que le pases la data a usar, x e y serían las columnas con las que se quiere trabajar.
#Y k sería las repeticiones para hacer el bootstrap.
def bootstrap (data, x, y, k): 
  n = len(data)                    #Para tener el tamaño de mi muestra.
  I_k = list(range(data.shape[0])) #Muestra D=(X,Y) de tamaño n indexadas con un variable I = list(range(n))
  b_k = []                         #En esta lista vacía voy a guardar mis coeficientes
  #Las siguientes listas van a guardar los quantiles de cada coeficiente para despues poder armar mi dataframe con todos los datos
  q1=[]
  q2=[]
  q3=[]
  q4=[]
  q5=[]
  for i in range(k):              #For para bootstrapear
    sampleo = np.random.choice(I, size=n) #1) Samplear con repetición de I un vector I*_k de tamaño n
    d_k = data.iloc[sampleo]              #2) Elegir las I_k filas de D para generar un data set D_k (de tamaño n)
    x_=pd.DataFrame(d_k["x"])             #3) Usar D_k para calcular un modelo de regresión f al que llameremos M_k
    y_=pd.DataFrame(d_k["y"])
    lm=linear_model.LinearRegression()
    m_k=lm.fit(X=x_, y=y_)
    coeficiente=m_k.coef_[0][0] #4) Extraer de M_k los coeficientes del modelo al cual llamaremos b_k, y le indico dos veces la posicion 0 para poder obtener mi coeficiente del array
    b_k.append(coeficiente)
    q1.append(coeficiente.quantile(0.025)) #Asi saco todos los cuantiles para el coeficiente producido
    q2.append(coeficiente.quantile(0.25))
    q3.append(coeficiente.quantile(0.5))
    q4.append(coeficiente.quantile(0.75))
    q5.append(coeficiente.quantile(0.9725))
  
  #Finalmente me queda un dataframe con todos los coeficientes extraidos y sus cuantiles
  coeficientes_y_cuantiles= pd.DataFrame({"Coeficientes":b_k, 
                                          "0.025":q1,
                                          "0.25":q2,
                                          "0.5":q3,
                                          "0.75":q4,
                                          "0.9725":q5})
  #Grafico de mis coeficientes
  plot.hist(data= coeficientes_y_cuantiles, x="Coeficientes",  color="Blue")
    
  return coeficientes_y_cuantiles

Y acá usé statsmodels

In [3]:
#Para hacer modelo de regresion usé https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html
def bootstrap (data, x, y, k): #dejo que las repeticiones k las elija la persona que use la función
  n = len(data)
  I= list(range(data.shape[0])) #Muestra D=(X,Y) de tamaño n indexadas con un variable I = list(range(n))
  b_k = [] #En esta lista vacía voy a guardar mis coeficientes
  #Las siguientes listas van a guardar los quantiles de cada coeficiente para despues poder armar mi dataframe con todos los datos
  q1=[]
  q2=[]
  q3=[]
  q4=[]
  q5=[]
  y_pred=[]
  for i in range(k):
    sampleo = np.random.choice(I, size=n) #1) Samplear con repetición de I un vector I*_k de tamaño n
    d_k = data.iloc[sampleo] #2) Elegir las I_k filas de D para generar un data set D_k (de tamaño n)
    y = d_k["y"]
    x = d_k["x"]
    x = sm.add_constant(x) 
    #construyo mi modelo
    modelo = sm.OLS(y,x)
    resultados = modelo.fit()
    coeficiente = resultados.params[1] #para obtener mi coeficiente de x
    b_k.append(coeficiente)
    q1.append(coeficiente.quantile(0.025)) #Asi saco todos los cuantiles para el coeficiente producido
    q2.append(coeficiente.quantile(0.25))
    q3.append(coeficiente.quantile(0.5))
    q4.append(coeficiente.quantile(0.75))
    q5.append(coeficiente.quantile(0.9725))
    #Ahora para cada iteracion debo predecir el valor de y_k con el modelo que obtengo en cada iteracion
    y_pred.append(resultados.predict(d_k['x']))
      
  #Finalmente me queda un dataframe con todos los coeficientes extraidos y sus cuantiles
  coeficientes_y_cuantiles= pd.DataFrame({"Coeficientes":b_k, 
                                          "0.025":q1,
                                          "0.25":q2,
                                          "0.5":q3,
                                          "0.75":q4,
                                          "0.9725":q5})
  

  return coeficientes_y_cuantiles, y_pred